<a href="https://colab.research.google.com/github/MjunMJ/python_playground/blob/cryptopals.com-challenges/cryptopals_set1_Q6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I took inspiration from [here](https://dev.to/wrongbyte/cryptography-basics-breaking-repeated-key-xor-ciphertext-1fm2), and [here](https://www.southampton.ac.uk/~wright/1001/index-of-coincidence.html) to figure out the method for question 6. Here is my reasoning, please leave a comment or DM me if there's something wrong with it as I am a noob in cryptography.

To get multi character XOR decipher, we need to guess the key length, and then we can use some kind of brute force method to get the key.

**Readable vs random line of words**

Based on articles I've read above, and other helpful online content, we need to understand the differences between a readable sentence, and one made up of random characters:

*   Letter frequency: A readable sentence is likely to have letters that occur at a known frequency, based on frequency data such as that [here](https://pi.math.cornell.edu/~mec/2003-2004/cryptography/subs/frequencies.html), where some letters are more likely to occur than others. A randomly constructed sentence however, has each character randomly picked, so each character has equal chance of appearing.
*   Hamming distance: Similar to the above, the hamming distance of a different parts of a readable sentence *should* be less than that of a randomly generated sentence. It doesn't seem to always happen in my random trials, and also as this blogger shares [here](https://trustedsignal.blogspot.com/2015/06/xord-play-normalized-hamming-distance.html), but it's a signal nonetheless. E.g. we have two strings, one legible, the other plain nonsense:


```
#hamming distance for every n bytes of a string

from itertools import combinations

def hamming_every_n_bytes (string,n):
  total=0
  count=0
  parts = [string[i:i+n] for i in range(0, len(string), n)]
  combo =combinations(parts,2)
  for part1, part2 in combo:
    xor=bytes([a ^ b for a, b in zip(bytes(part1, "ascii"), bytes(part2, "ascii"))])
    return sum(bin(byte).count('1') for byte in xor)/n

print(hamming_every_n_bytes("cats and dogs make great pets. please take care of them", 5)) #=2.4
print(hamming_every_n_bytes("ye93ocjexzmqs&sivabi0skw[df8]u58nslfvkuyuujdsfuefusoeqm", 5)) #=3.6
```

**Guessing the key length**

OK so the article suggest we use Hamming distance to guess the key length. Above, we guess this value is small when text is readable, but ciphertext is never readable. So how is hamming distance gonna help? I puzzled over this, and finally supposed this was the reasoning:
* Since the text was encoded with XOR, the hamming distance of the ciphertext is same as that of text. Here's why:

```
text = 11001100 01001001 (hamming =3)
key  = 00001001 00001001
xor  = 11000101 01000000 (hamming =3)
```

XOR swtiches a 0 to 1, and vice versa. So if you switch two sets of bits with the same XOR, the differences in the number of 1s remains unchanged.

In [3]:
#Get started by getting the cipher text

import base64
from itertools import combinations
import requests

url = "https://cryptopals.com/static/challenge-data/6.txt"

response = requests.get(url)
data = response.text
data_bytes = base64.b64decode(data)


In [4]:
#get possible key sizes now by finding the shorter hamming distances
#get hamming distance for one keysize
def hamming_every_n_bytes (string,n):
  total=0
  count=0
  parts = [string[i:i+n] for i in range(0, len(string), n)]
  combo =combinations(parts,2)
  for part1, part2 in combo:
    xor=bytes([a ^ b for a, b in zip(bytes(part1, "ascii"), bytes(part2, "ascii"))])
    return sum(bin(byte).count('1') for byte in xor)/n

dict_keysizes_hd ={}

#get hamming distance for many keysizes, narrow down to top 10
for k in range(2,40):
  dict_keysizes_hd[k] = hamming_every_n_bytes(data,k)

dict_top_10=sorted(dict_keysizes_hd.items(),key=lambda x:x[1])[0:9]

**Guessing the key**

After narrowing down to the top 10 keysizes based on hamming distance, we need to test the key, going through 256 potential byte for each one. So if the keysize I want to check is 10, there are 256 to the power of 10 possibilities. OMG

Cryptopals suggests that we "transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on." For each block, the nth letter is compared with the nth letter of our key. So if for each nth letter of our key, we can single XOR with all the nth letter of all the blocks, we can derive the key.

In [6]:
#break the ciphertext into blocks of length=keysize and transpose every n part with key in n position
from itertools import zip_longest

def transpose_blocks(ciphertext, keysize):
  blocks = [ciphertext[i:i+keysize] for i in range(0, len(ciphertext), keysize)]
  zipped = [list(filter(None,i)) for i in zip_longest(*blocks)]
  key=[]
  for z in zipped:
    character=xor_decipher_unknown_key(z)
    key.append(character)
  return (''.join(k for k in key))

#now go through each key in dict_top_10 to find the most human readable key.
#I think we are assuming the key is also human readable.
all_possible_keys = []

for d in dict_top_10:
  keysize_to_test = d[0]
  possible_key = bytes(transpose_blocks(data_bytes, keysize_to_test),"ascii")
  if possible_key is not None:
    all_possible_keys.append(possible_key)

In [8]:
#let's apply mult-character XOR now to data_bytes using all_possible_keys.
#we then count letter frequency to guess the key

letters_to_count = ['e','t','a','o','i','n','s','h','r','d','l','u','c']
max_count=0

for key in all_possible_keys:
  count_for_key=0
  decipher_text=multi_xor_known_key(data_bytes, key).decode("utf-8")
  for w in decipher_text:
    if w in letters_to_count: count_for_key+=1
  if count_for_key>max_count:
    max_count=count_for_key
    best_key=key

print(best_key)

#decipher data with best_key
print(multi_xor_known_key(data_bytes, best_key).decode("utf-8"))


b'TERMINATOR\x00X\x1a\x00BRING\x00THE\x00NOISE'
IM BACK ANd i'M RINGIN THE bELL *a ROCkIN' ON THE MIKE WhILE THE FLy GiRLS YELL *iN EcSTASY IN ThE bACK OF ME *wELl THATS MY dj dESHAY CUTTIN ALL THEM z'S 
hITTIN HARD AnD THE GIRLiES GOIN CRAZY *vaNILLAS ON THe MIKE MAN iM NOT LAZY 
*i'M LETTIN MY DrUG KICK IN *it CONTROLS MY MoUTH AND i bEGiN *tO JUST LET IT FLOW LeT mY CONCEPTS GO 
 kNoCK ON SOME WOOd *fOR GOOD LUcK i LIKE MY RhYMES ATROCiOUs *sUPERCALAFRAgILISTICEXPiALiDOCIOUS *iM An EFFECT ANd ThAT YOU CAN BET *i CAN TAKe A FLY GIRL AND MaKE HER WET. *
 oh MY gOD HoMEbODY YOU PROBAbLY EAT *sPaGHeTTI WITH A SPOoN cOME ON ANd SAY IT **vip. vANILLA icE yEP YEP iM CoMIN HARD lIKe A RHINO *iNTOxICATING SO YOu STAGGER LIKE a WINO *sO pUNkS STOP TRYING aND GIRL SToP cRYIN *vANILLA iCE IS SELlIN' AND YOU PEOPLe ARE BUYIN' *'cAUSE WHY THE fREAKS ARE jOCkIN LIKE cRAZY gLUE *mOVIn aND GROOVIN TRyING TO SINg AlONG *aLL THROUgH THE GHETtO gROOVIN THIS HeRE SONG *noW yOU